<a href="https://colab.research.google.com/github/simon-m-mudd/random_fun_notebooks/blob/main/gilt_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import date, timedelta
from math import isfinite

# -----------------------------
# Utilities
# -----------------------------
def add_months(d, months):
    """Add (or subtract) calendar months from a date, keeping end-of-month logic."""
    m = d.month - 1 + months
    y = d.year + m // 12
    m = m % 12 + 1
    # clamp day to last day of target month
    from calendar import monthrange
    last_day = monthrange(y, m)[1]
    return date(y, m, min(d.day, last_day))

def halfyear_backwards_schedule(maturity, freq=2):
    """Generate coupon dates stepping backwards in exact semiannual increments from maturity."""
    assert freq == 2, "This helper assumes semiannual coupons."
    dates = [maturity]
    # generate sufficiently far back (e.g., 60 years worth to be safe)
    for _ in range(240):
        dates.append(add_months(dates[-1], -6))
    return dates  # descending from maturity backwards

def next_and_prev_coupon_dates(settlement, maturity):
    """Find the previous and next coupon dates around settlement, assuming semiannual schedule."""
    # Build descending schedule from maturity (maturity, maturity-6m, maturity-12m, ...)
    sched_desc = halfyear_backwards_schedule(maturity, freq=2)
    # Find first coupon strictly after settlement as next, and the one before or equal as previous
    next_c = None
    prev_c = None
    for i in range(len(sched_desc)-1, -1, -1):
        c = sched_desc[i]
        if c > settlement:
            next_c = c
            # previous is the next element in descending order (i+1) if exists
            prev_c = sched_desc[i+1] if i+1 < len(sched_desc) else None
            break
        elif c == settlement:
            # On coupon date: previous is c, next is the one 6 months after
            prev_c = c
            next_c = sched_desc[i-1] if i-1 >= 0 else None
            break
    # If settlement is after all generated dates (shouldn't happen for realistic inputs), fallback
    if next_c is None:
        next_c = maturity
        prev_c = add_months(maturity, -6)
    return prev_c, next_c

def accrued_interest_act_act_icma(settlement, last_coupon, next_coupon, annual_coupon_rate, face=100, freq=2):
    """Accrued interest using Actual/Actual (ICMA) for a regular semiannual period."""
    if last_coupon is None or next_coupon is None:
        return 0.0
    coupon_amount = face * annual_coupon_rate / freq
    days_accrued = (settlement - last_coupon).days
    days_period = (next_coupon - last_coupon).days
    if days_period <= 0:
        return 0.0
    return coupon_amount * (days_accrued / days_period)

def build_remaining_cashflows(settlement, maturity, annual_coupon_rate, face=100, freq=2,
                              apply_ex_div=False, ex_div_days=7):
    """
    Remaining positive cashflows (date, amount) strictly after settlement.
    If apply_ex_div=True, exclude the next coupon if settlement falls within ~ex_div_days before it.
    """
    # Generate all coupon dates forward from the previous coupon after settlement
    # We construct via backward schedule and then filter > settlement.
    all_coupons_desc = halfyear_backwards_schedule(maturity, freq=freq)
    coupon_dates = sorted([d for d in all_coupons_desc if d > settlement])

    # Approximate ex-dividend logic (calendar-day approximation)
    if apply_ex_div and coupon_dates:
        next_coupon = coupon_dates[0]
        if (next_coupon - settlement).days <= ex_div_days:
            # Exclude the first coupon
            coupon_dates = coupon_dates[1:]

    coupon_amount = face * annual_coupon_rate / freq
    cashflows = [(d, coupon_amount) for d in coupon_dates]

    # Redemption at maturity (add principal)
    # If maturity appears in coupon_dates, add principal on that date
    found_mat = any(d == maturity for d, _ in cashflows)
    if found_mat:
        # add redemption on same date
        cashflows.append((maturity, face))

    return sorted(cashflows, key=lambda x: x[0])

def xnpv(rate, cashflows):
    """
    NPV for irregular cashflows: list of (date, amount), using Actual/365 day basis (Excel-like XIRR).
    First cashflow date is the time origin.
    """
    if not cashflows:
        return 0.0
    d0 = cashflows[0][0]
    total = 0.0
    for d, amt in cashflows:
        days = (d - d0).days
        total += amt / ((1.0 + rate) ** (days / 365.0))
    return total

def xirr(cashflows, guess=0.03, tol=1e-10, max_iter=200):
    """
    Solve for IRR using bisection with bracketing. Returns annualized rate.
    cashflows: list of (date, amount) with at least one negative and one positive amount.
    """
    # Bracket search
    lo, hi = -0.9999, 10.0
    f_lo = xnpv(lo, cashflows)
    f_hi = xnpv(hi, cashflows)

    # Expand hi if needed
    tries = 0
    while f_lo * f_hi > 0 and tries < 20:
        hi *= 2.0
        f_hi = xnpv(hi, cashflows)
        tries += 1

    if f_lo * f_hi > 0:
        # Fallback: try smaller hi ladder
        hi_candidates = [0.5, 1.0, 2.0, 5.0, 10.0]
        bracketed = False
        for h in hi_candidates:
            f_hi = xnpv(h, cashflows)
            if f_lo * f_hi <= 0:
                hi = h
                bracketed = True
                break
        if not bracketed:
            raise ValueError("Unable to bracket IRR; check cashflows.")

    # Bisection
    for _ in range(max_iter):
        mid = (lo + hi) / 2.0
        f_mid = xnpv(mid, cashflows)
        if abs(f_mid) < tol or abs(hi - lo) < tol:
            return mid
        if f_lo * f_mid <= 0:
            hi, f_hi = mid, f_mid
        else:
            lo, f_lo = mid, f_mid

    return mid  # best effort

# -----------------------------
# Main driver
# -----------------------------
def gilt_total_return(settlement, maturity, annual_coupon_rate, price,
                      price_is_clean=True, face=100.0, freq=2,
                      apply_ex_div=False, ex_div_days=7):
    """
    Compute total return (to maturity) and annualized IRR for a UK conventional gilt.

    Parameters
    ----------
    settlement : datetime.date
        Settlement (purchase) date.
    maturity : datetime.date
        Maturity date.
    annual_coupon_rate : float
        Annual coupon rate as a decimal (e.g., 0.045 for 4.5%).
    price : float
        Observed price per 100 nominal. Clean (default) or dirty as specified.
    price_is_clean : bool
        If True, 'price' is clean and accrued interest is added to form dirty price.
    face : float
        Nominal amount. Defaults to 100.
    freq : int
        Coupon frequency (semiannual=2 for UK gilts).
    apply_ex_div : bool
        Approximate ex-dividend behavior: exclude the next coupon if within 'ex_div_days'.
        NOTE: This uses calendar days, not UK business days; for precise handling, integrate a business-day calendar.
    ex_div_days : int
        Length (days) of approximate ex-div period.

    Returns
    -------
    dict with:
        - 'accrued_interest'
        - 'dirty_price'
        - 'cashflows' (list of (date, amount))
        - 'total_cash_inflows'
        - 'total_return_pct' (cumulative)
        - 'xirr_annual' (annualized IRR using exact dates)
    """
    assert freq == 2, "Function assumes semiannual coupons for UK conventional gilts."

    # Accrued interest (Actual/Actual ICMA on regular semiannual period)
    prev_cpn, next_cpn = next_and_prev_coupon_dates(settlement, maturity)
    ai = accrued_interest_act_act_icma(settlement, prev_cpn, next_cpn, annual_coupon_rate, face=face, freq=freq)

    dirty = price + ai if price_is_clean else price

    # Remaining cashflows (after settlement)
    cashflows_pos = build_remaining_cashflows(
        settlement, maturity, annual_coupon_rate, face=face, freq=freq,
        apply_ex_div=apply_ex_div, ex_div_days=ex_div_days
    )

    total_in = sum(amt for _, amt in cashflows_pos)

    # Cashflow list for IRR: negative outflow at settlement (dirty price), then inflows
    cf = [(settlement, -dirty)] + cashflows_pos
    irr = xirr(cf) if any(amt > 0 for _, amt in cf) and any(amt < 0 for _, amt in cf) else float('nan')

    total_return = (total_in - dirty) / dirty

    return {
        "accrued_interest": ai,
        "dirty_price": dirty,
        "cashflows": cashflows_pos,
        "total_cash_inflows": total_in,
        "total_return_pct": total_return,
        "xirr_annual": irr,
    }

# -----------------------------
# Example usage
# -----------------------------
if __name__ == "__main__":
    # Example (edit these values):
    settlement = date(2025,10, 22)             # your purchase/settlement date
    maturity   = date(2026,6, 22)              # gilt maturity date (e.g., 7 Sep 2034)
    coupon     = 0.015                          # 4.5% annual coupon
    clean_px   = 98.41                          # clean price per 100 nominal

    result = gilt_total_return(
        settlement, maturity, coupon, clean_px,
        price_is_clean=True, face=100.0, freq=2,
        apply_ex_div=False  # set True if you know you bought in the ex-div window
    )

    print("Accrued interest: ", round(result["accrued_interest"], 6))
    print("Dirty price:      ", round(result["dirty_price"], 6))
    print("Total inflows:    ", round(result["total_cash_inflows"], 6))
    print("Total return:     ", round(100 * result["total_return_pct"], 4), "%")
    print("Annualized XIRR:  ", round(100 * result["xirr_annual"], 6), "%")
    print("Remaining CFs:")
    for d, a in result["cashflows"]:
        print(f"  {d.isoformat()}  {a:.6f}")

Accrued interest:  0.5
Dirty price:       98.91
Total inflows:     101.5
Total return:      2.6185 %
Annualized XIRR:   3.981628 %
Remaining CFs:
  2025-12-22  0.750000
  2026-06-22  0.750000
  2026-06-22  100.000000


In [ ]:
from datetime import date
from calendar import monthrange

def add_months(d, months):
    """Add/subtract calendar months with end-of-month handling."""
    m = d.month - 1 + months
    y = d.year + m // 12
    m = m % 12 + 1
    last_day = monthrange(y, m)[1]
    return date(y, m, min(d.day, last_day))

def halfyear_backwards_schedule(maturity, freq=2):
    """Build coupon dates by stepping backwards in 6-month increments from maturity."""
    assert freq == 2, "UK conventional gilts use semiannual coupons."
    dates = [maturity]
    # Generate far enough back (e.g., 60 years worth)
    for _ in range(240):
        dates.append(add_months(dates[-1], -6))
    return dates  # descending (maturity, maturity-6m, ...)

def next_coupon_and_remaining(settlement, maturity, freq=2,
                              apply_ex_div=False, ex_div_days=7,
                              include_if_on_coupon_date=False):
    """
    Return the next scheduled coupon date and how many coupons remain until maturity.

    Parameters
    ----------
    settlement : date
        Your purchase/settlement date.
    maturity : date
        Gilt maturity date (also a coupon date).
    freq : int
        Coupon frequency; 2 for UK fixed gilts.
    apply_ex_div : bool
        If True, approximate ex-dividend behavior by excluding the next coupon when within
        'ex_div_days' prior to that coupon date (calendar-day approximation).
    ex_div_days : int
        Length in days for the ex-div window (approximate).
    include_if_on_coupon_date : bool
        If True and settlement is exactly on a coupon date, treat that coupon as upcoming.

    Returns
    -------
    dict with:
      - 'next_coupon_date_calendar': date | None
      - 'next_coupon_date_you_receive': date | None (considers ex-div if enabled)
      - 'remaining_coupon_dates': list[date]       (strictly after settlement, incl. maturity)
      - 'remaining_coupons_total': int
      - 'remaining_coupons_you_receive': int       (ex-div adjusted if enabled)
    """
    assert freq == 2, "This function assumes semiannual payments."

    if settlement >= maturity:
        return {
            "next_coupon_date_calendar": None,
            "next_coupon_date_you_receive": None,
            "remaining_coupon_dates": [],
            "remaining_coupons_total": 0,
            "remaining_coupons_you_receive": 0,
        }

    # Build the (descending) schedule and select those strictly after settlement
    desc_sched = halfyear_backwards_schedule(maturity, freq=freq)
    # If you want to include a coupon when settling on the coupon date:
    def is_future_coupon(d):
        return (d > settlement) or (include_if_on_coupon_date and d == settlement)

    remaining = sorted([d for d in desc_sched if is_future_coupon(d)])
    next_calendar = remaining[0] if remaining else None

    # Ex-dividend approximation (7 *business* days in the UK; here we use calendar days)
    next_you_receive = next_calendar
    remaining_you_receive = list(remaining)
    if apply_ex_div and next_calendar is not None:
        days_to_next = (next_calendar - settlement).days
        if days_to_next >= 0 and days_to_next <= ex_div_days:
            # You won't receive the very next coupon
            remaining_you_receive = remaining_you_receive[1:]
            next_you_receive = remaining_you_receive[0] if remaining_you_receive else None

    return {
        "next_coupon_date_calendar": next_calendar,
        "next_coupon_date_you_receive": next_you_receive,
        "remaining_coupon_dates": remaining,
        "remaining_coupons_total": len(remaining),
        "remaining_coupons_you_receive": len(remaining_you_receive),
    }

# -----------------------------
# Example usage
# -----------------------------
if __name__ == "__main__":
    # Example (edit these):
    settlement = date(2025, 10, 6)   # your purchase/settlement date
    maturity   = date(2034, 9, 7)    # e.g., 7 Sep 2034

    info = next_coupon_and_remaining(
        settlement, maturity,
        apply_ex_div=True,     # set False if you don't want ex-div approximation
        ex_div_days=7,         # UK gilts: 7 business days (approx here as 7 calendar days)
        include_if_on_coupon_date=False
    )

    print("Next coupon (calendar):   ", info["next_coupon_date_calendar"])
    print("Next coupon (you receive):", info["next_coupon_date_you_receive"])
    print("Remaining coupon dates:   ", [d.isoformat() for d in info["remaining_coupon_dates"]])
    print("Remaining coupons (total):", info["remaining_coupons_total"])


Next coupon (calendar):    2026-03-07
Next coupon (you receive): 2026-03-07
Remaining coupon dates:    ['2026-03-07', '2026-09-07', '2027-03-07', '2027-09-07', '2028-03-07', '2028-09-07', '2029-03-07', '2029-09-07', '2030-03-07', '2030-09-07', '2031-03-07', '2031-09-07', '2032-03-07', '2032-09-07', '2033-03-07', '2033-09-07', '2034-03-07', '2034-09-07']
Remaining coupons (total): 18
